In [77]:
import openai
import os
import pandas as pd
import numpy as np
#pd.set_option('display.max_colwidth', None)
pd.set_option("max_colwidth", None)
#pd.set_option("max_rows", None)

In [3]:
from dotenv import load_dotenv, find_dotenv
found = load_dotenv(find_dotenv())
if found:
    openai.api_key  = os.getenv('OPENAI_API_KEY')
else:
    print("couldn't find the key")

## Utility functions

In [50]:
def get_completion(transaction_prompt,
                   prompt_template,
                   system_prompt=None,
                   model="gpt-3.5-turbo",
                   max_tokens=250,
                   temperature=0,
                   top_p=1,
                   frequency_penalty=0,
                   presence_penalty=0):
    ''' openai chat completion object '''
    
    prompt_template = prompt_template.format(tran_desc=transaction_prompt)
    messages = [{"role": "user", "content": prompt_template}] 
    if system_prompt is not None:
        messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt_template},
            ]
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty
    )
    return response.choices[0].message["content"]

In [16]:
# adopted from https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

## Temporary sample data

In [11]:
df_ref = pd.read_csv('./100_labeled_consumer_transactions.csv')

In [95]:
for i in df_ref.sort_values(by='description', ascending=False).iterrows():
    if 'Wal' in i[1]['description'] or 'WAL' in i[1]['description']:
        print(i[0], i[1]['description'])

82 WAL-MART  Wal-  03/24 #000981637 PURCHASE                   WAL-MART  Wal-Mar  SAN DIEGO (C) CA


In [81]:
for i in df_ref.sort_values(by='description', ascending=False)[['description']]:
    print(i)
    if 'Wal' in i or 'WAL' in i:
        print(i)
        break

description


## PoC: get transaction category from transaction statement

In [47]:
system_prompt = "You are a financial assistant. You have to enrich transactions."

prompt_template = """
Given a bank transaction statement, extract the transaction category, Merchant and website.

Note: Use ' | ' as a seperator between transaction category and Merchant.

Examples:
Transaction: BEVERAGES & MOR GILROY CA 
Category: Food & Drink | Mechant: BevMo | website: bevmo.com
Transaction: EARLY PAY: PETCO ANIMAL SUP DIRECT DEP 661021916777KVV 012243
Category: Paycheck | Mechant: Petco | website: petco.com
Transaction: Acorns Investing Transfer 123980 JIKCR7 Cristiano Pombal
Category: Investment | Mechant: Aconrs | website: acorns.com
Transaction: USAA CREDIT CARD PAYMENT 9926283729231 WEB ID: 12987121
Category: Credit Card Bill | Merchant: USAA | usaa.com

user_prompt:
{tran_desc}
"""

sample_transaction_statement = 'BILL PAY BMW R18 CLASSIC RECURRING xxxxxx9876 ON 01-04'

In [51]:
get_completion(transaction_prompt= sample_transaction_statement,
                          prompt_template=prompt_template,
                          system_prompt=system_prompt,
                          model="gpt-3.5-turbo", max_tokens=500)

In [52]:
response

'Category: Bill Payment | Merchant: BMW | Website: N/A'

In [54]:
prompt_template = """
Given a bank transaction statement, extract the transaction category, Merchant and website.

Note: Use ' | ' as a seperator between transaction category and Merchant.

Examples:
Transaction: BEVERAGES & MOR GILROY CA 
Category: Food & Drink | Mechant: BevMo | website: bevmo.com
Transaction: EARLY PAY: PETCO ANIMAL SUP DIRECT DEP 661021916777KVV 012243
Category: Paycheck | Mechant: Petco | website: petco.com
Transaction: Acorns Investing Transfer 123980 JIKCR7 Cristiano Pombal
Category: Investment | Mechant: Aconrs | website: acorns.com
Transaction: USAA CREDIT CARD PAYMENT 9926283729231 WEB ID: 12987121
Category: Credit Card Bill | Merchant: USAA | usaa.com
Transaction: BILL PAY BMW R18 CLASSIC RECURRING xxxxxx9876 ON 01-04
Category: Auto Payment | Merchant: BMW | bmw.com

user_prompt:
{tran_desc}
"""

sample_transaction_statement = '0173 AMC MESQUITE 23 MESQUITE TX 01/06'

In [55]:
get_completion(transaction_prompt= sample_transaction_statement,
                          prompt_template=prompt_template,
                          system_prompt=system_prompt,
                          model="gpt-3.5-turbo", max_tokens=500)

'Category: Entertainment | Merchant: AMC | Website: amctheatres.com'

In [56]:
sample_transaction_statement = "Trader Joe's #012 Qps Sanfrancisco Ca"
get_completion(transaction_prompt= sample_transaction_statement,
                          prompt_template=prompt_template,
                          system_prompt=system_prompt,
                          model="gpt-3.5-turbo", max_tokens=500)

"Category: Food & Drink | Merchant: Trader Joe's | Website: traderjoes.com"

In [60]:
prompt_template = """
Given a bank transaction statement, extract the transaction category, Merchant and website.

Note: Use ' | ' as a seperator between transaction category and Merchant.

Examples:
Transaction: BEVERAGES & MOR GILROY CA 
Category: Food & Drink | Mechant: BevMo | website: bevmo.com
Transaction: EARLY PAY: PETCO ANIMAL SUP DIRECT DEP 661021916777KVV 012243
Category: Paycheck | Mechant: Petco | website: petco.com
Transaction: Acorns Investing Transfer 123980 JIKCR7 Cristiano Pombal
Category: Investment | Mechant: Aconrs | website: acorns.com
Transaction: USAA CREDIT CARD PAYMENT 9926283729231 WEB ID: 12987121
Category: Credit Card Bill | Merchant: USAA | usaa.com
Transaction: BILL PAY BMW R18 CLASSIC RECURRING xxxxxx9876 ON 01-04
Category: Auto Payment | Merchant: BMW | bmw.com
Transaction: Trader Joe's #012 Qps Sanfrancisco Ca
Category: Grocery | Merchant: Trader Joe's | traderjoes.com

user_prompt:
{tran_desc}
"""

In [61]:
sample_transaction_statement = "WAL-MART  Wal-  03/24 #000981637 PURCHASE"
get_completion(transaction_prompt= sample_transaction_statement,
                          prompt_template=prompt_template,
                          system_prompt=system_prompt,
                          model="gpt-3.5-turbo", max_tokens=500)

'Category: Retail | Merchant: Wal-Mart | Website: walmart.com'

In [68]:
df_ref.sort_values(by='description', ascending=False).head(1000)

,description,date,entry_type,amount,iso_currency_code,country,account_holder_type,account_holder_id,merchant_correct,website_correct,labels_correct
17,YSI*Allora Bella Terra TX 01/18,2023-01-18,outgoing,1527.70,USD,US,consumer,benchmark_account,Yardi,yardi.com,Rent and property management fee
2,Withdrawal ACH NSF Fee ($323.55 KWIKCASHKC),2023-01-03,outgoing,19.95,USD,US,consumer,benchmark_account,KwikCash,kwikcashonline.com,Non-sufficient funds / Overdraft fee
76,WIRE TYPE:WIRE IN DATE: 230317 TIME:1212 ET TRN:XXXXXXXXXX7991721 SEQ:XXXXX129812/0077821 ORIG:OKCOIN USA INC. ID:XXXXX221342 SND BK:SIGNATUR E BANK ID:XXXXX22671 PMT DET:XXXXXXXXXX78137182,2023-03-18,outgoing,47.25,USD,US,consumer,benchmark_account,Okcoin,okcoin.com,Trading (crypto)
23,WEIGELS 82 Q81,2023-01-24,outgoing,1.05,USD,US,consumer,benchmark_account,Weigel's,weigels.com,Convenience stores
24,WEBFLOW,2023-01-25,outgoing,18.90,USD,US,consumer,benchmark_account,Webflow,webflow.com,SaaS tools
...,...,...,...,...,...,...,...,...,...,...,...
13,23617 POS SIGNATURE FRITZS AUTO SPA LLC FRITZS AUTO SPA LL SAINT JOSEP,2023-01-14,outgoing,9.45,USD,US,consumer,benchmark_account,Fritz's Auto Spa,fritzscarwash.com,Vehicle maintenance
93,1ST BANKCARD CTR ONLINE PMT 331198 2F0C5DO189172 BRANDON J MIKE,2023-04-04,outgoing,74.12,USD,US,consumer,benchmark_account,First Bankcard,firstbankcard.com | card.fnbo.com,Credit card bill
4,1108914IH2NS48MR HABIT PROVO 0279,2023-01-05,outgoing,16.80,USD,US,consumer,benchmark_account,The Habit Burger Grill,habitburger.com,Food and Drink
29,0324 FOREIGN ATM FEE DAIGLS PHIL-K182913,2023-01-30,outgoing,2.10,USD,US,consumer,benchmark_account,NaN,NaN,Bank fee


In [ ]:
#step1: prompt engineering 

Query = self.Prompt_Query(Prompt_Template, Question, Verbose=True)

    # prompt engineering
    Prompt = self.Prompt_Question(Prompt_Template,{'{Question}':Question})
    def Prompt_Question(self, _Prompt_Template_, Inputs):
        """
        """
        for i,j in Inputs.items():
            Prompt = _Prompt_Template_.replace(i,j)
    return Prompt
    
    # get the input token prompt cost 
    Prompt_Cost(Prompt, self._LLM_Model, self._Token_Cost, self._Encoding_Base)
    
    # get response from openai
    Response = self.OpenAI_Completion(Prompt)
    
    # get output token cost
    

In [14]:
tran_desc

['EARLY PAY: PETCO ANIMAL SUP DIRECT DEP 661021916777KVV 012243',
 'DOORDASH*ZEEKS PIZZA',
 'Withdrawal ACH NSF Fee ($323.55 KWIKCASHKC)',
 'BILL PAY BMW R18 CLASSIC                       RECURRING xxxxxx9876 ON 01-04',
 '1108914IH2NS48MR HABIT PROVO 0279',
 '0173 AMC MESQUITE 23 MESQUITE TX 01/06',
 'SUDSHARE* S...876 SUDSHARE, .C #100',
 'EB SLANDER PRESEN 04/12 PURCHASE 8091735211',
 'Brigit-com PROTECTION WEB 8JDB682JD0',
 'Direct Debit: Napa Benefits, 892830232',
 'POS PURCH - AFTERPAY 5999 CA 101-928739273 AFTERPAY US',
 'TEDS MONTANA GRILL 12',
 'CAPITAL ONE ONLINE PMT 781123 2NYXE2URO2XY',
 '23617 POS SIGNATURE FRITZS AUTO SPA LLC FRITZS AUTO SPA LL SAINT JOSEP',
 'Goody Goody Liqu#10d',
 'WEAVER STREET MARKET H',
 'FROM CGB ENTERPRISE, DIRECT DEP',
 'YSI*Allora Bella Terra TX 01/18',
 'COSTA DEL SOL LATINO M HENRICO VA 01/19',
 'DSW PRESTONWOOD 5201 BELT',
 'DISHOOM BIRMINGHAM',
 'CHECKCARD 01/22 AFFIRM PAY 22LS5C0E',
 'PURCHASE                                AUTHORIZED ON   0

In [ ]:
def extract_response(self, pred: str) -> dict:
        pred = pred.replace(self.QUESTION_PROMPT, "").strip()
        if pred[-1] in [".", ";"]:  # Case when LLM add a punctuation point at the end
            pred = pred[:-1]
        # If output doesn't match the list of possible labels,
        # replace with "Not enough information"
        if pred not in self.ALL_LABELS:
            warnings.warn(
                f"LLM answer: '{pred}' not in label list, replacing by '{NOT_ENOUGH_INFO_LABEL}'."
            )
            pred = NOT_ENOUGH_INFO_LABEL
        return {"labels": pred}